# Russian Synodal Bible (1885) Text Processing

## Import

### Libraries

In [1]:
import os
import codecs
import pandas as pd 
import natasha
import re

### Text

In [11]:
texts = './texts/fiction/'
libCols = ['author','date','title','text']
tokenOHCO = ['title','part_num','para_num', 'sent_num', 'token_num']

In [35]:
libDf = pd.DataFrame(columns = libCols)
for t in os.listdir(texts): 
    if t[-4:] == '.txt': 
        #print(t)
        info = re.match(r'(\w+)-(\d{4})-(.+).txt', t)
        with codecs.open(texts+t, 'r', encoding='windows-1251') as f: 
            textytext = f.read()
        libDf = libDf.append({
            'author': info.group(1),
            'date': int(info.group(2)), 
            'title': info.group(3), 
            'text': textytext
        }, ignore_index=True)
        
libDf = libDf.sort_values(libCols[1:3]).set_index(libCols[2])
tokenDf = libDf[[libCols[3]]]
libDf = libDf.drop(columns=[libCols[3]])
libDf

,author,date
title,,
k-zvezdam,andreev,1905
tak-bylo,andreev,1905
mat,gorkii,1906
savva-ignis-sanat,andreev,1906
iuda-iskariot,andreev,1907
zhizn-cheloveka,andreev,1907
ispoved,gorkii,1908
zhizn-nenuzhnogo-cheloveka,gorkii,1908
gorodok-okurov,gorkii,1909


In [36]:
tokenDf = tokenDf.text.str.split('\n\n\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:2]
tokenDf = tokenDf.rename(columns={0:'parts'})
tokenDf = tokenDf.parts.str.split('\n\s*\n', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:3]
tokenDf = tokenDf.rename(columns={0:'paras'})
tokenDf = tokenDf.paras.str.split(r'!|\?|\.|;', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO[:4]
tokenDf = tokenDf.rename(columns={0:'sents'})
sentDf = tokenDf
tokenDf = tokenDf.sents.str.split(r'\n|\s+', expand=True).stack().to_frame()
tokenDf.index.names = tokenOHCO
tokenDf = tokenDf.rename(columns={0:'token'})
tokenDf = tokenDf.token.apply(lambda x: x.lower().strip(r"\W\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
tokenDf = tokenDf[tokenDf['token'].str.contains(r'\w+')]
tokenDf

token
title     part_num para_num sent_num token_num              
k-zvezdam 0        1        0        1          обсерватория
                                     2                     в
                                     3                 горах
                            1        1               поздний
                                     2                 вечер
...                                                      ...
igo-voiny 0        139      94       3                 плачу
                                     4                   все
                                     5                 плачу
                                     6                   все
                                     7                 плачу

[845144 rows x 1 columns]

In [37]:
sentDf = sentDf[sentDf['sents'].str.contains(r'\w+')].sents.apply(lambda x: x.lower().strip(r"\W\n\|\]\}\[\{\.\'\"\?;:,<>/1234567890")).to_frame()
sentDf[sentDf['sents'].str.contains('r\w')]

sents
title      part_num para_num sent_num                                                   
k-zvezdam  0        71       113       * вы видите надпись /(показывая на\n        /\...
                             114                              curae procul este profanae
                             116                                     hinc  itur ad astro
gubernator 16       1        186        развернул он ее на первых\n    страницах и до...
           24       1        181        в венеции я бывал, но в церковь maria\n    de...